<a href="https://colab.research.google.com/github/Mun09/Attention-is-all-you-need/blob/main/Attention_is_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 논문 제목: Attention Is All You Need
## 논문 구현 프로젝트
### 구현자: JaeYoung Moon

이 노트북은 논문 "Attention Is All You Need"의 구현을 다루며, 해당 논문의 주요 알고리즘과 KoreantoEnglish 번역실험을 재현합니다.


#마운팅

In [219]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##필수 라이브러리 설치

In [220]:
%%capture
# if device == 'cpu':
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
# else:
#   !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu117
!pip install torch
!pip install torchtext==0.13.1
!pip install spacy==3.3.0
!pip install pandas
!pip install sklearn

In [221]:
import pandas as pd

# ai-hub 한국어문장-영어문장 20만개
data_path = '/content/drive/MyDrive/dataset.xlsx'
data = pd.read_excel(data_path)

print(data.head)
print(data.shape)


<bound method NDFrame.head of            SID                                                 원문  \
0            1  'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1            2                                       씨티은행에서 일하세요?   
2            3              푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   
3            4   11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.   
4            5     6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.   
...        ...                                                ...   
199995  199996                               나는 먼저 청소기로 바닥을 밀었어요.   
199996  199997                             나는 먼저 팀 과제를 하고 놀러 갔어요.   
199997  199998                              나는 비 같은 멋진 연예인을 좋아해요.   
199998  199999                           나는 멋진 자연 경치를 보고 눈물을 흘렸어.   
199999  200000                               나는 멋진 중학교 생활을 기대합니다.   

                                                      번역문  
0       Bible Coloring' is a coloring application that...  
1                    

In [222]:
import os

n = data.shape[0]

if not os.path.isfile('/content/drive/MyDrive/dataset_ko.txt'):
  f = open('/content/drive/MyDrive/dataset_ko.txt', 'w', encoding='utf-8')
  for i in range(n):
    f.write(data.iloc[i, 1] + '\n')
  f.close()

if not os.path.isfile('/content/drive/MyDrive/dataset_en.txt'):
  f = open('/content/drive/MyDrive/dataset_en.txt', 'w', encoding='utf-8')
  for i in range(n):
    f.write(data.iloc[i, 2] + '\n')
  f.close()

In [223]:
import sentencepiece as spm

# 텍스트 토큰화 함수 정의
def tokenize_text(text, sp_processor):
    return sp_processor.encode_as_pieces(text)

# 토큰 데이터 반환
def make_token(file_path, save_path, token_model_prefix):

  with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
  if not os.path.isfile(save_path):

    with open(save_path, 'w', encoding='utf-8') as f:
      for line in lines:
        f.write(f'{line.strip()}\n')

    # SentencePiece 모델 학습
  spm.SentencePieceTrainer.train(input=save_path, model_prefix=token_model_prefix, vocab_size=32000)

  # SentencePiece 모델 로드
  sp = spm.SentencePieceProcessor()
  sp.load(token_model_prefix + '.model')

  return [tokenize_text(line.strip(), sp) for line in lines]

def make_index_list(word_list, vocab_file):
  with open(vocab_file, 'r', encoding='utf-8') as f:
      lines = f.readlines()

  # 각 토큰과 해당 인덱스를 가져오기
  token_to_index = {}
  index_to_token = {}
  for idx, line in enumerate(lines):
      token = line.split("\t")[0]  # 토큰은 탭으로 구분된 첫 번째 요소입니다.
      token_to_index[token] = idx
      index_to_token[idx] = token

  index_list = []
  start, end = token_to_index['<s>'], token_to_index['</s>']
  for word_list_item in word_list:
    here = [token_to_index.get(token, token_to_index['<unk>']) for token in word_list_item]
    here = [start] + here + [end]
    index_list.append(here)
  return token_to_index, index_to_token, index_list


In [224]:
file_path = '/content/drive/MyDrive/dataset_ko.txt'
save_path = '/content/drive/MyDrive/train_text_ko.txt'
token_model_prefix = 'spm_ko_32000'

token_ko = make_token(file_path, save_path, token_model_prefix)

file_path = '/content/drive/MyDrive/dataset_en.txt'
save_path = '/content/drive/MyDrive/train_text_en.txt'
token_model_prefix = 'spm_en_32000'

token_en = make_token(file_path, save_path, token_model_prefix)

In [225]:
token_to_index_ko, index_to_token_ko, index_list_ko = make_index_list(token_ko, 'spm_ko_32000.vocab')
token_to_index_en, index_to_token_en, index_list_en = make_index_list(token_en, 'spm_en_32000.vocab')

In [226]:
print(index_list_ko[4])
for i in index_list_ko[4]:
  print(index_to_token_ko[i])

print(index_list_en[4])
for i in index_list_en[4]:
  print(index_to_token_en[i])

[1, 8, 21737, 12, 429, 12, 476, 915, 9, 171, 11565, 40, 21212, 4620, 1483, 2402, 1512, 3, 2]
<s>
▁
6.5
,
▁7
,
▁8
▁사이즈
가
▁몇
▁개나
▁더
▁재입고
▁될지
▁제게
▁알려주시면
▁감사하겠습니다
.
</s>
[1, 5, 69, 144, 1967, 6, 62, 143, 114, 3760, 28, 26, 7715, 13, 510, 14115, 7, 9575, 12, 810, 3, 2]
<s>
▁I
▁would
▁feel
▁grateful
▁to
▁know
▁how
▁many
▁stocks
▁will
▁be
▁secured
▁of
▁size
▁6.5
,
▁7,
▁and
▁8
.
</s>


In [227]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# 토큰화된 데이터셋과 레이블 (예: ko_tokens, en_tokens)
X = index_list_ko  # 입력 데이터
y = index_list_en  # 레이블 데이터 (필요한 경우)

# 데이터셋 정의
class CustomDataset(Dataset):
    def __init__(self, src, trg=None):
        self.src = src
        self.trg = trg

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        if self.trg is not None:
            return self.src[idx], self.trg[idx]
        else:
            return self.src[idx]

In [228]:
def my_collate(batch):
    # batch는 (data, label) 튜플의 리스트입니다.
    # data와 label을 분리합니다.
    src, trg = zip(*batch)
    src = [torch.tensor(item) for item in src]
    trg = [torch.tensor(item) for item in trg]
    # src_trg = [torch.tensor(src), torch.tensor(trg)]
    # data를 패딩하여 동일한 길이로 맞춥니다.
    padding_value = 0
    padded_src = pad_sequence(src, batch_first=True, padding_value=padding_value)
    padded_trg = pad_sequence(trg, batch_first=True, padding_value=padding_value)

    return padded_src, padded_trg

In [229]:
train_size = 190000
valid_size = 5000

# 학습, 검증, 테스트 데이터로 나누기
train_data = CustomDataset(X[:train_size], y[:train_size])
valid_data = CustomDataset(X[train_size:train_size+valid_size], y[train_size:train_size+valid_size])
test_data =  CustomDataset(X[train_size+valid_size:], y[train_size+valid_size:])

# DataLoader 정의
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=my_collate)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, collate_fn=my_collate)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=my_collate)

# 데이터 확인
print("학습 배치 개수:", len(train_loader))
print("검증 배치 개수:", len(valid_loader))
print("테스트 배치 개수:", len(test_loader))


학습 배치 개수: 1485
검증 배치 개수: 40
테스트 배치 개수: 40


In [230]:
# 데이터 확인
for batch_idx, (data, labels) in enumerate(train_loader):
    # 각 배치의 데이터 크기 확인
    print("Batch", batch_idx)
    print("Data size:", data.size())  # (배치 크기, 시퀀스 최대 길이)
    print("Labels size:", labels.size())  # (배치 크기,)
    break

for batch_idx, (data, labels) in enumerate(valid_loader):
    # 각 배치의 데이터 크기 확인
    print("Batch", batch_idx)
    print("Data size:", data.size())  # (배치 크기, 시퀀스 최대 길이)
    print("Labels size:", labels.size())  # (배치 크기,)
    break

Batch 0
Data size: torch.Size([128, 29])
Labels size: torch.Size([128, 35])
Batch 0
Data size: torch.Size([128, 31])
Labels size: torch.Size([128, 39])


# Scaled Dot-Product Attention 아키텍처

#Multi-Head Attention 아키텍처




* attention 입력 요소
  *   query
  * keys
  *   values
  *   
*   hyperparameter
  * hidden_dim



In [231]:
import torch.nn as nn

device = torch.device('cuda')

In [232]:
class MultiHeadAttention(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio=0.1, device=device):
    super().__init__()

    assert(hidden_dim % n_heads == 0)

    self.hidden_dim = hidden_dim
    self.n_heads = n_heads
    self.n_heads_dim = hidden_dim // n_heads
    self.dropout_ratio = dropout_ratio
    self.device = device

    # linear_layer
    self.fc_q = nn.Linear(hidden_dim, hidden_dim).to(device)
    self.fc_k = nn.Linear(hidden_dim, hidden_dim).to(device)
    self.fc_v = nn.Linear(hidden_dim, hidden_dim).to(device)

    self.fc = nn.Linear(hidden_dim, hidden_dim).to(device)

    self.dropout = nn.Dropout(dropout_ratio).to(device)

    self.scale = torch.sqrt(torch.FloatTensor([self.n_heads_dim])).to(device)

  def forward(self, query, key, value, mask=None):
    batch_size = query.shape[0]

    # Q : [batch_size, query_len, hidden_dim]
    # K : [batch_size, key_len, hidden_dim]
    # V : [batch_size, value_len, hidden_dim]
    Q = self.fc_q(query).to(self.device)
    K = self.fc_k(key).to(self.device)
    V = self.fc_v(value).to(self.device)

    # Q : [batch_size, n_heads, query_len, n_heads_dim]
    # K : [batch_size, n_heads, key_len, n_heads_dim]
    # V : [batch_size, n_heads, value_len, n_heads_dim]
    Q = Q.view(batch_size, -1, self.n_heads, self.n_heads_dim).permute(0, 2, 1, 3).to(self.device)
    K = K.view(batch_size, -1, self.n_heads, self.n_heads_dim).permute(0, 2, 1, 3).to(self.device)
    V = V.view(batch_size, -1, self.n_heads, self.n_heads_dim).permute(0, 2, 1, 3).to(self.device)

    # energy : [batch_size, n_heads, query_len, key_len]
    energy = torch.matmul(Q, K.permute(0, 1, 3, 2)).to(self.device) / self.scale

    if mask is not None:
      energy = energy.masked_fill(mask == 0, -1e10).to(self.device)

    # attention : [batch_size, n_heads, query_len, key_len]
    attention = torch.softmax(energy, dim=-1).to(self.device)

    x = torch.matmul(self.dropout(attention), V).to(self.device)
    # x = [batch_size, n_heads, query_len, n_heads_dim]

    x = x.permute(0, 2, 1, 3).contiguous().to(self.device)
    x = x.view(batch_size, -1, self.hidden_dim).to(self.device)
    x = self.fc(x).to(self.device)
    # x = [batch_size, query_len, hidden_dim]

    return x ,attention

#Feedforward 아키텍처


*   간단하게 feedforward
*   입력 차원 = 출력차원



In [233]:
class FeedForward(nn.Module):
  def __init__(self, hidden_dim, fc_dim, dropout_ratio=0.1, device=device):
    super().__init__()
    self.device = device

    self.fc1 = nn.Linear(hidden_dim, fc_dim).to(device)
    self.fc2 = nn.Linear(fc_dim, hidden_dim).to(device)

    self.dropout = nn.Dropout(dropout_ratio).to(device)


  def forward(self, x):
    x = self.dropout(torch.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

#EncoderLayer 아키텍처

In [234]:
class EncoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, fc_dim, dropout_ratio=0.1, device=device):
    super().__init__()
    self.device = device

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim).to(device)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim).to(device)
    self.self_attn = MultiHeadAttention(hidden_dim, n_heads, dropout_ratio, device).to(device)
    self.ff = FeedForward(hidden_dim, fc_dim, dropout_ratio, device).to(device)
    self.dropout = nn.Dropout(dropout_ratio).to(device)

  def forward(self, src, mask):
    # src = [batch_size, src_len, hidden_dim]
    # mask = [batch_size, src_len]

    _out, _ = self.self_attn(src, src, src, mask=mask)
    out = self.self_attn_layer_norm(src + self.dropout(_out))
    _out = self.ff(out)
    out = self.ff_layer_norm(out + self.dropout(_out))
    #out = [batch_size, src_len, hidden_dim]

    return out

#Encoder 아키텍처

In [235]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, n_heads, fc_dim, dropout_ratio=0.1, max_length=100, device=device):
    super().__init__()
    self.device = device

    self.enc_embedding = nn.Embedding(input_dim, hidden_dim).to(device)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim).to(device)

    self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, fc_dim, dropout_ratio, device) for _ in range(n_layers)]).to(device)
    self.dropout = nn.Dropout(dropout_ratio).to(device)
    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, src, mask):
    batch_size = src.shape[0]
    src_len = src.shape[1]

    pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
    pos = self.pos_embedding(pos).to(self.device)

    src = self.dropout(self.enc_embedding(src) * self.scale + pos).to(self.device)
    # src = [batch_size, src_len, hidden_dim]

    for layer in self.layers:
      out = layer(src, mask)

    return out

#DecoderLayer 아키텍처

In [236]:
class DecoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, fc_dim, dropout_ratio=0.1, device=device):
    super().__init__()
    self.device = device

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim).to(device)
    self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim).to(device)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim).to(device)

    self.self_attn = MultiHeadAttention(hidden_dim, n_heads, dropout_ratio, device)
    self.enc_attn = MultiHeadAttention(hidden_dim, n_heads, dropout_ratio, device)
    self.ff = FeedForward(hidden_dim, fc_dim, dropout_ratio)

    self.dropout = nn.Dropout(dropout_ratio).to(device)



  def forward(self, trg, enc_out, src_mask, trg_mask):
    # trg : [batch_size, trg_len, hidden_dim]
    # enc_out : [batch_size, src_len, hidden_dim]
    # src_mask : [batch_size, src_len]
    # trg_mask : [batch_size, trg_len]

    _trg, _ = self.self_attn(trg, trg, trg, mask=trg_mask)
    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

    _trg, attention = self.enc_attn(trg, enc_out, enc_out, mask=src_mask)
    trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

    _trg = self.ff(trg)
    trg = self.ff_layer_norm(trg + self.dropout(_trg))

    return trg, attention

In [237]:
class Decoder(nn.Module):
  def __init__(self, output_dim, hidden_dim, n_layers, n_heads, fc_dim, dropout_ratio=0.1, max_length=100, device=device):
    super().__init__()
    self.device = device

    self.dec_embedding = nn.Embedding(output_dim, hidden_dim).to(device)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim).to(device)

    self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, fc_dim, dropout_ratio, device) for _ in range(n_layers)])
    self.fc = nn.Linear(hidden_dim, output_dim).to(device)
    self.dropout = nn.Dropout(dropout_ratio).to(device)
    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, trg, enc_out, src_mask, trg_mask):
    batch_size = trg.shape[0]
    trg_len = trg.shape[1]

    pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
    pos = self.pos_embedding(pos)

    trg = self.dropout(self.dec_embedding(trg) * self.scale + pos)
    # trg = [batch_size, trg_len, hidden_dim]

    for layer in self.layers:
      trg, attention = layer(trg, enc_out, src_mask, trg_mask)

    out = self.fc(trg)
    # trg = [batch_size, trg_len, output_dim]

    return out, attention

#Transformer 아키텍처

In [238]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device=device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx
    self.device = device

  def make_src_mask(self, src):
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2).to(self.device)
    # src_mask : [batch_size, 1, 1, src_len]

    return src_mask

  def make_trg_mask(self, trg):
    _, trg_len = trg.shape
    trg_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
    # trg_mask : [batch_size, 1, trg_len, trg_len]
    trg_mask_one = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2).to(self.device)
    trg_mask = trg_mask & trg_mask_one
    # trg_mask : [batch_size, 1, trg_len, trg_len]

    return trg_mask

  def forward(self, src, trg):
    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)
    # src_mask : [batch_size, 1, 1, src_len]
    # trg_mask : [batch_size, 1, trg_len, trg_len]

    enc_out = self.encoder(src, src_mask)
    # enc_out : [batch_size, src_len, hidden_dim]

    out, attention = self.decoder(trg, enc_out, src_mask, trg_mask)
    # out : [batch_size, trg_len, output_dim]
    # attention : [batch_size, n_heads, trg_len, src_len]

    return out, attention

#모델 training

In [239]:
INPUT_DIM = len(token_to_index_ko)
OUTPUT_DIM = len(token_to_index_en)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
MAX_LEGNTH=100 # 한 문장 단어 최대 개수

In [240]:
SRC_PAD_IDX = 0
TRG_PAD_IDX = 0

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, MAX_LEGNTH, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, MAX_LEGNTH, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [241]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 28,612,864 trainable parameters


In [242]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (enc_embedding): Embedding(32000, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attn): MultiHeadAttention(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): FeedForward(
          (fc1): Linear(in_features=256, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
    

In [243]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch[0].to(model.device)
        trg = batch[1].to(model.device)

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [244]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch[0].to(model.device)
            trg = batch[1].to(model.device)

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [245]:
import torch.optim as optim
# import torch._dynamo

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

#학습 시작

In [246]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [247]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')
cnt = 0

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_korean_to_english.pt')
        cnt=0
    else:
      cnt += 1

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

    if cnt == 2:
      break

Epoch: 01 | Time: 3m 36s
	Train Loss: 4.081 | Train PPL: 59.181
	Validation Loss: 2.730 | Validation PPL: 15.339
Epoch: 02 | Time: 3m 35s
	Train Loss: 2.819 | Train PPL: 16.760
	Validation Loss: 2.290 | Validation PPL: 9.873
Epoch: 03 | Time: 3m 34s
	Train Loss: 2.316 | Train PPL: 10.133
	Validation Loss: 2.131 | Validation PPL: 8.422
Epoch: 04 | Time: 3m 34s
	Train Loss: 1.997 | Train PPL: 7.367
	Validation Loss: 2.066 | Validation PPL: 7.892
Epoch: 05 | Time: 3m 34s
	Train Loss: 1.767 | Train PPL: 5.850
	Validation Loss: 2.061 | Validation PPL: 7.855
Epoch: 06 | Time: 3m 34s
	Train Loss: 1.593 | Train PPL: 4.919
	Validation Loss: 2.082 | Validation PPL: 8.018
Epoch: 07 | Time: 3m 36s
	Train Loss: 1.459 | Train PPL: 4.302
	Validation Loss: 2.108 | Validation PPL: 8.232


#평가

In [248]:
# from google.colab import files

# files.download('transformer_korean_to_english.pt')

In [249]:
sentence = "우리 모두 오늘 하루 기분 좋게 생활해봐요."
sp = spm.SentencePieceProcessor()
sp.Load("spm_ko_32000.model")

tokens = sp.EncodeAsPieces(sentence)
token_idx = [token_to_index_ko[token] for token in tokens]
token_idx = [0] + token_idx + [0]
print(token_idx)

[0, 23, 166, 52, 364, 1018, 2617, 647, 11893, 32, 3, 0]


In [256]:
# 번역(translation) 함수
def translate_sentence(sentence, tokeninzer_model, model, device=device, max_len=50, logging=True):
  model.eval() # 평가 모드

  sp = spm.SentencePieceProcessor()
  sp.Load(tokeninzer_model + ".model")

  # 문장을 subword 단위로 토크나이즈
  tokens = sp.EncodeAsPieces(sentence)
  src_indexes = [token_to_index_ko[token] for token in tokens]
  src_indexes = [token_to_index_ko['<s>']] + src_indexes + [token_to_index_ko['</s>']]

  print(src_indexes)
  print(1)

  src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

  # 소스 문장에 따른 마스크 생성
  src_mask = model.make_src_mask(src_tensor)

  # 인코더(endocer)에 소스 문장을 넣어 출력 값 구하기
  with torch.no_grad():
      enc_src = model.encoder(src_tensor, src_mask)

  # 디코더의 첫번째 입력으로 사용할 <sos>
  trg_indexes = [token_to_index_en['<s>']]

  # 첫번째 디코더 입력으로

  for i in range(max_len):
      trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

      # 출력 문장에 따른 마스크 생성
      trg_mask = model.make_trg_mask(trg_tensor)

      with torch.no_grad():
          output, attention = model.decoder(trg_tensor, enc_src, src_mask, trg_mask)

      # 출력 문장에서 가장 마지막 단어만 사용
      pred_token = output.argmax(2)[:,-1].item()

      if pred_token in [token_to_index_en['</s>']]:
        break
      trg_indexes.append(pred_token) # 출력 문장에 더하기

      # <eos>를 만나는 순간 끝


  # 각 출력 단어 인덱스를 실제 단어로 변환
  trg_tokens = [index_to_token_en[i] for i in trg_indexes]

  # 첫 번째 <sos>는 제외하고 출력 문장 반환
  return trg_tokens[1:], attention

In [251]:
%%capture
!pip install ipywidgets

In [254]:
import ipywidgets as widgets
from IPython.display import display, clear_output

text_input = widgets.Text(
    value='',
    placeholder='Enter a sentence',
    description='Sentence:',
    disabled=False
)

button = widgets.Button(
    description='영어로 번역하기',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)

def on_button_click(b):
    sentence = text_input.value
    translation, attn = translate_sentence(sentence, "spm_ko_32000", model, device)
    clear_output(wait=True)
    display(text_input, button)
    print("모델 출력 결과:", " ".join(translation))

button.on_click(on_button_click)
display(text_input, button)

Text(value='저는 지금 굉장히 배고프고 밥을 먹기를 원하는 상태입니다.', description='Sentence:', placeholder='Enter a sentence')

Button(description='영어로 번역하기', icon='check', style=ButtonStyle(), tooltip='Click me')

모델 출력 결과: ▁I ' m ▁very ▁hungry ▁now , ▁and ▁I ' m ▁going ▁to ▁eat ▁something .
